In [ ]:
#import connection services to google big query and connect
from google.cloud import bigquery
from google.oauth2 import service_account

service_path = "C:\\Users\\hauer\\OneDrive - The University of Montana\\MSBA\\BMKT 670 Applied Data Analytics\\Wedge Project\\"
service_file = 'hauer-ada-wedgeproject-2894572d63a8.json' # this is your authentication information  
gbq_proj_id = 'hauer-ada-wedgeproject'  # change this to your project_id
gbq_dataset_id = 'wedge_full' # and change this to your data set ID

private_key = service_path + service_file

# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
# set a query for GBQ that asks for unique owners who are not equal to 3 - the code for nonowners
query = (
    "SELECT DISTINCT card_no "
    "FROM `wedge_full.transArchive_clean` "
    "WHERE card_no != 3 "
)

# execute the query in GBQ with `client.query`
results = client.query(
    query,
    location="US",
)

In [ ]:
# view the number of items returned by the query
len(list(results))

In [ ]:
# view the first 10 results of the query
list(results)[:10]

In [ ]:
# import the python random pacage to randomly sample the results of the owner query
import random

# set the owners value to empty
owners = []

# add sampled owner to the owners list
for owner in results :
    owners.append(owner[0])

# random sample owners from the query
sample = random.choices(owners, k=475)
new_sample = list(sample)

# show the sampled owners
print(sample)

In [ ]:
# create a new query for GBQ using the random sample of owners
query = (
    "SELECT * "
    "FROM `wedge_full.transArchive_clean` "
    "WHERE card_no in ( "
)

for card in sample:
    query += str(card) + ", "
query = query[:-2] + ')'

# execute the query in GBQ with `client.query`
results = client.query(
    query,
    location="US",
)

In [ ]:
# set up headers for the text file with the data returned from the sampled owners query
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

# create and save the data returned from the sampled owners into a text file with the headers
with open('sample_owners_trans.txt', 'w') as outfile :
    outfile.write(','.join(headers) + "\n") 
    for line in results :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")